In [1]:
import os
import fnmatch
import subprocess
import re
import simpleMenu
import ConfigParser
import logging
from logging.handlers import RotatingFileHandler
import sys

appShortName = 'fileStreamPortfolio'

In [ ]:
configPath = os.path.expanduser('~/.config/'+appShortName)
configFile = configPath+'/config.ini'
print configFile
config = ConfigParser.ConfigParser()
config.read(configFile)
config.add_section('Main')
try:
    with open (configFile, 'wb') as myFile:
        config.write(myFile)
except IOError:
    try:
        os.makedirs(configPath)
    except Exception as e:
        print e

In [ ]:
configfile = './fileStreamPortfolio.ini'
config = ConfigParser.ConfigParser()
config.read(configfile)

try:
    config.add_section('Main')
    config.add_section('Recent')
except ConfigParser.DuplicateSectionError as e:
    pass

config.set('Main', 'DefaultMountPoint', '/Volumes/GoogleDrive')
config.set('Main', 'TeamDrive', '')
config.set('Main', 'PortfolioPath', '')

config.set('Recent', 'GradeFolder', '')

try:
    myConfig = open(configfile, 'wb')
    config.write(myConfig)
except Exception as e:
    print e

In [8]:
class configuration(object):
    def __init__(self, configPath = os.path.expanduser('~/.config/'+appShortName)):
        self.logger = logging.getLogger(__name__)
        self.configPath = configPath
        self.configFile = os.path.join(configPath, 'config.ini')
        self.getConfig()
    
    def writeConfig(self):
        # write out all the defined preferences (self.prefs) to the config file
        self.logger.debug('writing configuration to file: {}'.format(self.configFile))
        for key in self.prefs:
            eval ("self.parser.set('{0}', '{1}', self.{1})".format(self.mainSection, key))
        try:
            self.parser.write(open(self.configFile, 'w'))
        except Exception as e:
            logging.error('Error writing configuration file: {}'.format(e))
    
#     def printConfig(self):
#         for key in self.prefs:
#             print key, eval('self.{0}'.format(key))
    
    def getConfig(self):
        '''
        Reads configuration file and sets the following attributes:
        '''
        
        self.parser = ConfigParser.SafeConfigParser()
        
        # required options in the 'Main' section
        self.mainSection = 'Main'
        
        # required key: [method for getting, default value]
        self.prefs = {
                        'mountpoint': [self.parser.get, '/Volumes/GoogleDrive'],
                        'teamdrive': [self.parser.get, ''],
                        'lastgradefolders': [self.parser.get, './gradefolders.txt'],
                        'portfoliofolder': [self.parser.get, '']
        }

        # make sure a configuration path exists
        if len(self.parser.read(self.configFile)) <= 0:
            logging.warn('no configuration files found at: {}'.format(self.configFile))
            logging.debug('creating configuration files')
            try:
                os.makedirs(os.path.expanduser(self.configPath))
            except OSError as e:
                if e.errno != 17:
                    logging.critical(e)
                    sys.exit(1)
        
        # make sure there is a main section
        if not self.parser.has_section(self.mainSection):
            self.parser.add_section(self.mainSection)
        
        preferences = {}
        
        # read search for the expected preferences in the configuration file
        # note which are missing and set to the default values above
        for key in self.prefs:
            try:
                preferences[key] = self.prefs[key][0](self.mainSection, key)
            except (ConfigParser.NoSectionError, ConfigParser.NoOptionError):
                self.parser.set(self.mainSection, key, self.prefs[key][1])
                preferences[key] = self.prefs[key][1]
                
        # set the values from the config
        for key in self.prefs:
            exec ("self.{0} = preferences['{0}']".format(key))
        
#         print preferences
#         print 'teamdrive', self.teamdrive
#         print 'mountpoint', self.mountpoint
#         print 'portfoliofolder', self.portfoliofolder
#         print 'lastgradefolders', self.lastgradefolders

In [ ]:
myConfig = configuration(configPath = './'+appShortName)

# myConfig.writeConfig()
# myConfig.mountpoint
myConfig.printConfig()

In [ ]:
myConfig.portfoliofolder


In [ ]:
foo = "foo"
eval(foo)

In [3]:
class teamDrives(object):
    '''
    make working with google Team Drives through filestream a little easier
    '''

    def __init__(self, mountpoint='/Volumes/GoogleDrive/Team Drives/'):
        self.logger = logging.getLogger(__name__)
        self.mountpoint = mountpoint
        self.getDrives()
    
    def getDrives(self):
        self.logger.debug('Searching for Google Drive File Stream Mount Points')
        self.drives = {}
        try:
            drives = next(os.walk(self.mountpoint))[1]
        except Exception as e:
            self.logger.critical('error retriving list of Team Drives: {0}'.format(e))
            self.logger.critical('is Google Drive File Stream application running and configured?')
            self.logger.critical('mount point: {} is not accessible'.format(self.mountpoint))
            self.logger.critical('exiting')
            raise os.error('Error in os.walk for mount point: {0}'.format(self.mountpoint))
        for drive in drives:
            self.drives[drive] = oct(os.stat(self.mountpoint+drive).st_mode & 0o777)
    
    def listrwDrives(self):
        rwDrives = []
        for drive in self.drives:
            if 777 - int(self.drives[drive]) < 277:
                rwDrives.append(drive)
        return(sorted(rwDrives))
    
    def listFolders(self, teamDrive):
        try:
            folders = next(os.walk(self.mountpoint+teamDrive))[1]
        except Exception as e:
            self.logger.critical('Error getting list of Team Drives: {0}'.format(e))
            self.logger.critical('Is Google Drive File Stream application running and configured?')
            raise os.error('Error in os.walk for mount point: {0}'.format(self.mountpoint))
        return(folders)
    
    def find(self, pattern, teamDrive):
        result = []
        for root, dirs, files in os.walk(self.mountpoint+teamDrive):
            for name in dirs:
                if fnmatch.fnmatch(name, pattern):
                    result.append(os.path.join(root, name))
        return result
                
                
    

        

In [4]:
def checkFSMount(mountpoint = '/Volumes/GoogleDrive'):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    logger.debug('Searching for Google Drive File Stream Mount Points')
    # list the local partitions (including FUSE partitions)
    mount = subprocess.check_output(['df', '-hl'])
    mountLines = mount.split('\n')
    mountPoints = []
    
    # makek a list of all the partitions
    for line in mountLines:
        try:
            # re extract anything that looks like a mount point
            mountSearch = re.search('\s+(\/[\S+]{0,})$', line)
            mountPoints.append(mountSearch.group(1))
        # ignore anything that doesn't match the re
        except Exception:
            pass    
    # check for mount point and try to launch the google drive file stream app
    if mountpoint in mountPoints:
        logger.debug('Found what appears to be a valid mountpoint at: {0}'.format(mountpoint))
        return True
    else:
        logger.info('Google Drive File Stream appears to not be running')
        return False
    


In [ ]:
def ltest(i = 'abcdefg'):
    logger = logging.getLogger(__name__)
    logger.debug('stuff: {0}'.format(i))
    logger.info('other stuff')
    logger.critical('important stuff')
    #logging.warn('stuff: {0}'.format(i))

In [13]:
def main():

    #### TESTING VARIABLES #####
    # remove these!
    cfgfile = './'+appShortName
    # //end remove
    #### TESTING VARIABLES #####
    
    # Create the Logger
    logger = logging.getLogger(__name__)

    for each in range (0, len(logger.handlers)):
        logger.removeHandler(logger.handlers[0])

    datefmt = '%y-%m-%d %H:%M:%S'

    logger.setLevel(logging.INFO)

    # file handler
#     fileformat = '%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s'
#     file_handler = logging.FileHandler(appShortName+'.log')
#     file_handler.setLevel(logging.DEBUG)
#     file_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))

    # stream handler
    streamformat = '%(asctime)s %(levelname)s - %(funcName)s: %(message)s'
    stream_handler = logging.StreamHandler(sys.stderr)
    stream_handler.setLevel(logging.CRITICAL)
    stream_handler.setFormatter(logging.Formatter(fmt = streamformat, datefmt = ''))

    # rotation handler
    fileformat = '%(asctime)s %(levelname)s - %(funcName)s: %(message)s'
    rotation_handler = RotatingFileHandler(appShortName+'.log', maxBytes = 50000, backupCount = 5)
    rotation_handler.setLevel(logging.DEBUG)
    rotation_handler.setFormatter(logging.Formatter(fmt = fileformat, datefmt = datefmt))


    # add handler to logger
    #logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    logger.addHandler(rotation_handler)
    logger.info('===================== Starting Log =====================')
        
    if checkFSMount():
        pass
    else:
        logger.info('Attempting to start Google Drive File Stream')
        try:
            gDriveFS = subprocess.check_call(["open", "-a", "Google Drive File Stream"])
        except subprocess.CalledProcessError as err:
            logger.warn('OS Error: {0}'.format(err))
            logger.critical('Google Drive File Stream does not appear to be installed. Please download from the link below')            
            logger.critical('https://support.google.com/drive/answer/7329379?hl=en')
            logger.critical('exiting')
            return(0)
        if checkFSMount():
            pass
        else:
            print "exiting"
            return(0)

    # get the configuration file
    myConfig = configuration(cfgfile)
    logger.info('=== Current Configuration Settings ===')
    for key in myConfig.prefs:
        try:
            logger.info('{0} -- {1}'.format(key, eval('myConfig.{0}'.format(key))))
        except Exception as e:
            pass
    
    myDrives = teamDrives()
    
    if not myConfig.teamdrive:
        logger.info('no teamdrive set in configuration file')
        # get a list of the avialable team drives
        try:
            myDrives.getDrives()
        except Exception as e:
            logger.critical(e)
            return(0)
        if len(myDrives.listrwDrives()) < 1:
            logger.critical('No Team Drives with write permissions available; exiting')
            return(0)
        else:
            rwDrivesMenu = simpleMenu.menu(name = 'Team Drives', items = myDrives.listrwDrives())
   
    
    if not myConfig.portfoliofolder:
        logger.info('no portfolio folder set in configuration file')
        # move this into a def
        try:
            myDrives.getDrives()
        except Exception as e:
            logger.critical(e)
            return(0)
        if len(myDrives.listrwDrives()) < 1:
            logger.critical('No Team Drives with write permissions available; exiting')
            return(0)
        else:
            rwDrivesMenu = simpleMenu.menu(name = 'Team Drives', items = myDrives.listrwDrives())
        
        myTeamDrive = rwDrivesMenu.loopChoice(optional = True, message = 'Which Team Drive contains the portfolio folder?')

        folderSearch = raw_input('Please enter part of the portfolio folder name (case sensitive search): ')
        foldersMenu = simpleMenu.menu(name = 'Matching Folders', items = myDrives.find(pattern = '*'+folderSearch+'*', teamDrive = myTeamDrive))
        myFolder = foldersMenu.loopChoice(optional = True, message = 'Which folder contains portfolios?')
        myConfig.portfoliofolder = myFolder
        
        if myFolder is 'Q':
            print 'exiting'
            return(0)
    
    continueMenu = simpleMenu.menu(name = 'Continue', items = ['Yes', 'No'])
    response = continueMenu.loopChoice(optional = True, message = 'Continue with the portfolio folder: {0}{1}'
                            .format(myConfig.teamdrive, myConfig.portfoliofolder))
    if response is 'Yes':
        logger.info('using portfolio folder: {0}{1}'.format(myConfig.teamdrive, myConfig.portfoliofolder))
        myConfig.writeConfig()
    if response is 'No':
        #need to do something about this; make some defs for get teamdrive and portofolio folder 
        pass
    if response is 'Q':
        print 'exiting'
        return(0)
    
    
    # open the student data file and start creating folders as needed

    
main()

Continue with the portfolio folder: IT Blabla/Volumes/GoogleDrive/Team Drives/IT Blabla/c/b/a/Portfolios
===== Continue =====
 1) Yes
 2) No
 1 - 2 or {'Q': 'Quit'}: 2

